In [1]:
import pandas as pd
from glob import glob
import numpy as np

In [2]:
df= pd.read_csv('/clusterfs/jgi/groups/gentech/homes/romalley/full_DAPseq_annotation/raw_data/N4_filtered-annotated-peaks_minfoldch5_genebody_plus_500bpStart-to-plus-600bpStop_111924.tsv', sep='\t', header=0)
df = df.loc[df['species']=="Arabidopsis_thaliana_Col-0"]

In [3]:
#reading TF label df
labels = pd.read_csv('/clusterfs/jgi/groups/gentech/homes/romalley/full_DAPseq_annotation/raw_data/ath-258-tf-info_simple.csv', sep=',', header=0)

# counting number of members of each TF family
fam_count = labels['tf_family'].value_counts().reset_index(drop=False)
fam_count= pd.DataFrame(fam_count).rename(columns={"index": "tf_family", "count": "fam_count"})

labels_count = labels.merge(fam_count, on = 'tf_family')
labels_count.head()

,gene_id,tf_family,tf_name,tf_and_fam,representative_category,tf_clade_curated,clade,fam_count
0,AT5G60130,ABI3VP1,AT5G60130,ABI3VP1_AT5G60130,1,2,ABI3VP1_c2,3
1,AT4G01500,ABI3VP1,NGA4,ABI3VP1_NGA4,0,1,ABI3VP1_c1,3
2,AT3G18990,ABI3VP1,VRN1,ABI3VP1_VRN1,2,3,ABI3VP1_c3,3
3,AT5G64750,AP2EREBP,ABR1,AP2EREBP_ABR1,2,1,AP2EREBP_c1,43
4,AT1G01250,AP2EREBP,AT1G01250,AP2EREBP_AT1G01250,0,1,AP2EREBP_c1,43


In [4]:
# adding tf info to large table 
dfl = df.merge(labels_count, left_on = 'tf', right_on = "gene_id", how = "left")
dfl.head()

,peak_chr,peak_start,peak_end,peak_name,peak_foldch,peak_pscore,peak_qscore,peak_summit,feature_chr,feature_source,...,n_cons_species_minfrac0,n_species_in_orthogroup,gene_id,tf_family,tf_name,tf_and_fam,representative_category,tf_clade_curated,clade,fam_count
0,Chr3,15645684,15646225,AT1G01060_A_thaliana_Col-0_DAPi709D10-ORGi5005...,121.6020,1156.460,1152.010,268,Chr3,phytozomev12,...,1,3,AT1G01060,MYBRELATED,LHY,MYBRELATED_LHY,1,3,MYBRELATED_c3,21
1,Chr3,10891839,10892318,AT1G01060_A_thaliana_Col-0_DAPi709D10-ORGi5005...,118.3000,1117.450,1113.010,250,Chr3,phytozomev12,...,1,4,AT1G01060,MYBRELATED,LHY,MYBRELATED_LHY,1,3,MYBRELATED_c3,21
2,Chr3,10449558,10450060,AT1G01060_A_thaliana_Col-0_DAPi709D10-ORGi5005...,118.1170,1115.290,1110.850,252,Chr3,phytozomev12,...,4,4,AT1G01060,MYBRELATED,LHY,MYBRELATED_LHY,1,3,MYBRELATED_c3,21
3,Chr3,10450281,10450581,AT1G01060_A_thaliana_Col-0_DAPi709D10-ORGi5005...,19.4416,103.225,100.876,148,Chr3,phytozomev12,...,4,4,AT1G01060,MYBRELATED,LHY,MYBRELATED_LHY,1,3,MYBRELATED_c3,21
4,Chr4,2117267,2117827,AT1G01060_A_thaliana_Col-0_DAPi709D10-ORGi5005...,110.4140,1025.150,1020.760,285,Chr4,phytozomev12,...,1,4,AT1G01060,MYBRELATED,LHY,MYBRELATED_LHY,1,3,MYBRELATED_c3,21


In [5]:
# creating bins to classify summit_to_cds_start
l1 = list(range(-5000, 600, 50))
l2 = list(range(-4950, 650, 50))

bins_df = pd.DataFrame(zip(l1, l2), columns = ['start', 'end'])

bins_df['mid'] = (bins_df['start'] + bins_df['end'])/2

bins_df.head()

,start,end,mid
0,-5000,-4950,-4975.0
1,-4950,-4900,-4925.0
2,-4900,-4850,-4875.0
3,-4850,-4800,-4825.0
4,-4800,-4750,-4775.0


In [ ]:
## the following part is ofr making plot using stop codon as origin

In [6]:
def dist_bins(summit_to_cds_end):
    for index, row in bins_df.iterrows():
        if (summit_to_cds_end >= row['start']) and (summit_to_cds_end < row['end']):
            return(row['mid'])
       # if (summit_to_cds_end < -5000):
       #        #     return(-4975.0)


# running function over every row
dfl['s2cds_bin'] = dfl['summit_to_cds_end'].apply(lambda x: dist_bins(x))
display(dfl)


,peak_chr,peak_start,peak_end,peak_name,peak_foldch,peak_pscore,peak_qscore,peak_summit,feature_chr,feature_source,...,n_species_in_orthogroup,gene_id,tf_family,tf_name,tf_and_fam,representative_category,tf_clade_curated,clade,fam_count,s2cds_bin
0,Chr3,15645684,15646225,AT1G01060_A_thaliana_Col-0_DAPi709D10-ORGi5005...,121.60200,1156.4600,1152.01000,268,Chr3,phytozomev12,...,3,AT1G01060,MYBRELATED,LHY,MYBRELATED_LHY,1,3,MYBRELATED_c3,21,525.0
1,Chr3,10891839,10892318,AT1G01060_A_thaliana_Col-0_DAPi709D10-ORGi5005...,118.30000,1117.4500,1113.01000,250,Chr3,phytozomev12,...,4,AT1G01060,MYBRELATED,LHY,MYBRELATED_LHY,1,3,MYBRELATED_c3,21,275.0
2,Chr3,10449558,10450060,AT1G01060_A_thaliana_Col-0_DAPi709D10-ORGi5005...,118.11700,1115.2900,1110.85000,252,Chr3,phytozomev12,...,4,AT1G01060,MYBRELATED,LHY,MYBRELATED_LHY,1,3,MYBRELATED_c3,21,-1025.0
3,Chr3,10450281,10450581,AT1G01060_A_thaliana_Col-0_DAPi709D10-ORGi5005...,19.44160,103.2250,100.87600,148,Chr3,phytozomev12,...,4,AT1G01060,MYBRELATED,LHY,MYBRELATED_LHY,1,3,MYBRELATED_c3,21,-425.0
4,Chr4,2117267,2117827,AT1G01060_A_thaliana_Col-0_DAPi709D10-ORGi5005...,110.41400,1025.1500,1020.76000,285,Chr4,phytozomev12,...,4,AT1G01060,MYBRELATED,LHY,MYBRELATED_LHY,1,3,MYBRELATED_c3,21,-875.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939080,Chr1,16683223,16683453,AT5G67580_A_thaliana_Col-0_DAPi711D05-ORGi5005...,5.09331,10.2649,8.02015,149,Chr1,phytozomev12,...,2,AT5G67580,MYBRELATED,TRB2,MYBRELATED_TRB2,0,1,MYBRELATED_c1,21,425.0
939081,Chr1,16755013,16755199,AT5G67580_A_thaliana_Col-0_DAPi711D05-ORGi5005...,5.09331,10.2649,8.02015,75,Chr1,phytozomev12,...,3,AT5G67580,MYBRELATED,TRB2,MYBRELATED_TRB2,0,1,MYBRELATED_c1,21,-1125.0
939082,Chr5,14070207,14070396,AT5G67580_A_thaliana_Col-0_DAPi711D05-ORGi5005...,5.09331,10.2649,8.02015,72,Chr5,phytozomev12,...,4,AT5G67580,MYBRELATED,TRB2,MYBRELATED_TRB2,0,1,MYBRELATED_c1,21,-3075.0
939083,Chr5,19938064,19938200,AT5G67580_A_thaliana_Col-0_DAPi711D05-ORGi5005...,5.07075,10.5667,8.32021,79,Chr5,phytozomev12,...,4,AT5G67580,MYBRELATED,TRB2,MYBRELATED_TRB2,0,1,MYBRELATED_c1,21,-25.0


In [8]:
cds_bin_count = dfl.groupby(["tf", 's2cds_bin', 'n_cons_species_minfrac0'])['peak_name'].agg('count').reset_index()
cds_bin_count = pd.DataFrame(cds_bin_count).rename(columns={"peak_name": "s2cds_peak_bin_count"})

peak_count = dfl.groupby(["tf", 'n_cons_species_minfrac0'])['peak_name'].agg('count').reset_index().rename(columns={"peak_name": "tf_by_cons_peak_count"})
peak_count_fam = dfl.groupby(["tf"])['peak_name'].agg('count').reset_index().rename(columns={"peak_name": "alltf_peak_count"})

display(peak_count_fam)

display(peak_count)


,tf,alltf_peak_count
0,AT1G01060,2996
1,AT1G01250,1434
2,AT1G01720,1988
3,AT1G02230,10087
4,AT1G02250,11780
...,...,...
239,AT5G66300,5654
240,AT5G66700,4657
241,AT5G66940,6345
242,AT5G67300,2694


,tf,n_cons_species_minfrac0,tf_by_cons_peak_count
0,AT1G01060,1,1075
1,AT1G01060,2,1042
2,AT1G01060,3,624
3,AT1G01060,4,255
4,AT1G01250,1,478
...,...,...,...
970,AT5G67300,4,381
971,AT5G67580,1,406
972,AT5G67580,2,241
973,AT5G67580,3,116


In [10]:
peak_count = peak_count.merge(peak_count_fam, on= ['tf'], how = 'left')
bin_count = cds_bin_count.merge(peak_count, on = ['tf', 'n_cons_species_minfrac0'], how = "left")

bin_count

,tf,s2cds_bin,n_cons_species_minfrac0,s2cds_peak_bin_count,tf_by_cons_peak_count,alltf_peak_count
0,AT1G01060,-4975.0,1,1,1075,2996
1,AT1G01060,-4925.0,2,1,1042,2996
2,AT1G01060,-4925.0,3,2,624,2996
3,AT1G01060,-4825.0,2,1,1042,2996
4,AT1G01060,-4825.0,3,1,624,2996
...,...,...,...,...,...,...
84806,AT5G67580,475.0,3,1,116,816
84807,AT5G67580,525.0,1,17,406,816
84808,AT5G67580,525.0,2,4,241,816
84809,AT5G67580,525.0,3,1,116,816


In [11]:
bin_count['s2cds_prop'] = bin_count['s2cds_peak_bin_count'] / bin_count['tf_by_cons_peak_count'] 
bin_count['tf_pcount'] = bin_count['tf'] + "_" +  bin_count['alltf_peak_count'].astype(str)  
display(bin_count)

,tf,s2cds_bin,n_cons_species_minfrac0,s2cds_peak_bin_count,tf_by_cons_peak_count,alltf_peak_count,s2cds_prop,tf_pcount
0,AT1G01060,-4975.0,1,1,1075,2996,0.000930,AT1G01060_2996
1,AT1G01060,-4925.0,2,1,1042,2996,0.000960,AT1G01060_2996
2,AT1G01060,-4925.0,3,2,624,2996,0.003205,AT1G01060_2996
3,AT1G01060,-4825.0,2,1,1042,2996,0.000960,AT1G01060_2996
4,AT1G01060,-4825.0,3,1,624,2996,0.001603,AT1G01060_2996
...,...,...,...,...,...,...,...,...
84806,AT5G67580,475.0,3,1,116,816,0.008621,AT5G67580_816
84807,AT5G67580,525.0,1,17,406,816,0.041872,AT5G67580_816
84808,AT5G67580,525.0,2,4,241,816,0.016598,AT5G67580_816
84809,AT5G67580,525.0,3,1,116,816,0.008621,AT5G67580_816


In [12]:
len(bin_count['tf'].unique())

244

In [13]:
# saving table 
bin_count.to_csv('/clusterfs/jgi/groups/gentech/homes/romalley/full_DAPseq_annotation/analysis/distance_CDS_bin_prop_by_tf_cons.tsv' , sep='\t', index=False)


In [6]:
## make plot using TTS as origin

def dist_bins(summit_to_cds_end):
    for index, row in bins_df.iterrows():
        if (summit_to_cds_end >= row['start']) and (summit_to_cds_end < row['end']):
            return(row['mid'])
       # if (summit_to_cds_end < -5000):
       #        #     return(-4975.0)


# running function over every row
dfl['s2cds_bin'] = dfl['summit_to_transcript_end'].apply(lambda x: dist_bins(x))

cds_bin_count = dfl.groupby(["tf", 's2cds_bin', 'n_cons_species_minfrac0'])['peak_name'].agg('count').reset_index()
cds_bin_count = pd.DataFrame(cds_bin_count).rename(columns={"peak_name": "s2cds_peak_bin_count"})

peak_count = dfl.groupby(["tf", 'n_cons_species_minfrac0'])['peak_name'].agg('count').reset_index().rename(columns={"peak_name": "tf_by_cons_peak_count"})
peak_count_fam = dfl.groupby(["tf"])['peak_name'].agg('count').reset_index().rename(columns={"peak_name": "alltf_peak_count"})


peak_count = peak_count.merge(peak_count_fam, on= ['tf'], how = 'left')
bin_count = cds_bin_count.merge(peak_count, on = ['tf', 'n_cons_species_minfrac0'], how = "left")

bin_count

bin_count['s2cds_prop'] = bin_count['s2cds_peak_bin_count'] / bin_count['tf_by_cons_peak_count'] 
bin_count['tf_pcount'] = bin_count['tf'] + "_" +  bin_count['alltf_peak_count'].astype(str)  

# saving table 

bin_count.to_csv('/clusterfs/jgi/groups/gentech/homes/romalley/full_DAPseq_annotation/analysis/distance_TTS_bin_prop_by_tf_cons.tsv' , sep='\t', index=False)